In [1]:
import random
import datetime

In [13]:
class Student:
    """Student class"""
    
    def __init__(self, name, section, motivation):
        self.name = name
        self.section = section
        self.motivation = motivation
        self.successes = []
    
    def attempt(week):
        effort = random.randrage(10) + self.motivation
        if (effort > 10):
            # due date - 3 days +- 12hourrange
            effort
        elif (effort > 5):
            # due date - 12hourrange
            effort
        elif (effort > 2):
            # due date + 6hourrange
            effort
        else:
            # due date + 6days +- 24 hourrange
            effort
            

class Course:
    """Course class"""

    def __init__(self, sections, length):
        self.sections = sections
        self.length = length
        self.students = []
        
        # Create students
        for section in sections:
            for i in range(section.get("size")):
                name = str(random.randrange(1000000,9999999))
                motivation = random.randrange(3)
                self.students.append(
                    Student(name, section.get("label"), motivation)
                )

    def run(self, file = True):
        "simulates running a semester of the course"
        # create submission table
        # each student makes Course.length attempts on an assignment
        # output submission table as file and return its path

In [ ]:
def pick_time(target, attempt):
    if (attempt == "early"):
        hours = random.randint(-12,12)
        datetime.timedelta(hours=)

## Intended Flow 


```python
course = Course(sections, 16)
submissions = course.run()

gradebook = grade(submissions)
```

### `sections`
`sections` should be a `list` of `dict`s that are all of the form:
```python
AD1 = {
    "label":"AD1",
    "size": 12,
    "start": "2019-01-14-12"
}
```

In [19]:
str(random.randrange(1000000,9999999))

'8140190'

In [42]:
AD1 = {
    "label":"AD1",
    "size": 3,
    "start": "2019-01-14-12"
}

AD2 = {
    "label":"AD2",
    "size": 4,
    "start": "2019-01-14-13"
}

test_course = Course([AD1, AD2], 3)

In [45]:
[print(s.section) for s in test_course.students];

AD1
AD1
AD1
AD2
AD2
AD2
AD2


In [11]:
try (test2):
except NameError:
    print("False")

SyntaxError: invalid syntax (<ipython-input-11-021d0870144f>, line 1)

In [9]:
test = []
test

IndexError: list index out of range